In [1]:
import pandas as pd
import numpy as np

In [ ]:
#-------------------------------------特征选择的类型-----------------------------------------
#一、 基于统计的特征选择(相关系数法；p值法)
data.corr()
import seaborn as sns
import matplotlib.style as style
style.use('fivethirtyeight')
sns.heatmap(data.corr())

In [ ]:
data.corr()['Y'] #特征和响应变量之间的相关性
bool_index=data.corr()['Y'].abs()>0.2 #过滤器，找出相关系数大于0.2的bool索引
highly_correlated_features=data.columns[bool_index] #存储与响应变量高度相关的特征
highly_correlated_features=highly_correlated_features.drop('Y') #找出特征变量，删除响应变量
X_subsetted=X[highly_correlated_features]
get_best_model_and_accuracy(DT,tree_params,X_subsetted,Y)

In [ ]:
#将相关性特征选择方法封装
from sklearn.base import TransformerMixin,BaseEstimator
class CustomCorrelationChooser(TransformerMixin,BaseEstimator):
    def __init__(self,response,cols_to_keep=[].threshold=None):
        self.response=response #保存响应变量
        self.threshold=threshold #保存阈值
        self.cols_to_keep=cols_to_keep #初始化一个变量，存放要保留的特征名
    
    def transform(self,X):
        return X[self.cols_to_keep] #转换会选择合适的列
    
    def fit(self,X):
        df=pd.concat([X,self.response],axis=1) #创建新的Dataframe，存放特征和响应
        self.cols_to_keep=df.columns[df.corr()[df.columns[-1]].abs>self.threshold] #保留高于阈值的列的名称
        self.cols_to_keep=[c for c in self.cols_to_keep if c in X.columns] #只保留X的列，删除响应变量
        return self
#实例化    
ccc=CustomCorrelationChooser(threshold=0.2,response=Y)
ccc.fit_transform(X)

In [10]:
#将特征选择方法加入pipeline中
from copy import deepcopy
#使用响应变量初始化特征选择器
ccc=CustomCorrelationChooser(response=Y)
#创建流水线，包括选择器
ccc_pipe=Pipeline([('correlation_select',ccc),('classifier',DT)])
tree_pipe_params={'classifier_max_depth':[None,1,3,5,7,9,11,13,15,17,19]}
#复制决策树参数
tree_pipe_params=deepcopy(tree_pipe_params)
#更新决策树的参数选择
ccc_pipe_params.update({'correlation_select_threshold':[0,0.1,0.2,0.3]})
#查看效果
get_best_model_and_accuracy(ccc_pipe,ccc_pipe_params,X,Y)

In [ ]:
#2.使用假设检验选择特征
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
k_best=SelectKBest(f_classif,k=5)
k_best.fit_transform(X,Y)
k_best.pvalues_
p_values=pd.DataFrame({'columns':X.columns,'p_value':k_best.pvalues_}).sort_values('p_value')
#bool索引
p_values['p_values']<0.05
p_values[p_values['p_values']<0.05]

In [ ]:
#用SelectKbest建立流水线
select_k_pipe=Pipeline([('k_best',k_best),('classifier',DT)]) #建立流水线
select_k_best_pipe_params=deepcopy(tree_pipe_params) 
select_k_best_pipe_params.update({'k_best_k':list(range(1,23))+['all']})
print(select_k_best_pipe_params)
#查看效果
get_best_model_and_accuracy(select_k_pipe,select_k_best_pipe_params,X,Y)
#查看选择的特征
k_best=SelectKBest(f_classif,k=7)

In [ ]:
#-----------------------二、基于模型的特征选择-----------------
#使用机器学习选择特征(拟合一个决策树，从而输出特征重要性，然后进行筛选)
tree=DecisionTreeClassifier()
tree.fit(X,Y)
importances=pd.DataFrame({'importance':tree.feature_importances_,'feature':X.columns}).sort_values('importance',ascending=False)
from sklearn.feature_selection import SelectFromModel
select_from_model=SelectFromModel(DecisionTreeClassifier(),threshold=0.05)
selected=select_from_model.fit_transform(X,Y)

In [ ]:
tree_pipe_params={'classifier_max_depth':[1,3,5,7,9]}
from sklearn.pipeline import Pipeline
#创建基于DecisionTreeClassifier的SelectFromModel
select=SelectFromModel(DecisionTreeClassifier)
select_from_pipe=Pipeline([('select',select),('classifier',DT)])
select_from_pipe_params=deepcopy(tree_pipe_params)
select_from_pipe_params.update({'select_threshold':[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,'mean','median','2.*mean'],'select_estimator_max_depth':[None,1,3,5,7,9]})
print(select_from_pipe_params)
get_best_model_and_accuracy(select_from_pipe,select_from_pipe_params,X,Y)


In [ ]:
#设置流水线最佳参数
select_from_pipe.set_params(**{'select_threshold':0.01,
                              'select_estimator_max_depth':None,
                              'classifier_max_depth':3})
select_from_pipe.steps[0][1].fit(X,Y)
X.columns[select_from_pipe.steps[0][1].get_support()]